In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
fits_file = "eclipsing_binaries/tess-s0005-1-1_65.170600_-1.740100_100x100_astrocut.fits"
# fits_file = "tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"

In [ ]:
eb = tess_cpm.Source(fits_file, remove_bad=True)

In [ ]:
eb.set_aperture(rowrange=[49, 53], colrange=[49, 53])
eb.plot_cutout(rowrange=[40, 60], colrange=[40, 60], show_aperture=True)

In [ ]:
eb.plot_pix_by_pix("normalized_flux")

In [ ]:
eb.add_cpm_model()
eb.set_regs([0.01])
eb.holdout_fit_predict(k=10);

In [ ]:
eb.plot_pix_by_pix("normalized_flux", split=True)

In [ ]:
lc = eb.get_aperture_lc("cpm_subtracted_flux")
plt.plot(eb.time, lc)

# outliers = np.full(eb.time.shape, False)
# while True:
#     sigma = np.std(lc[~outliers])
#     out = np.abs(lc) > 4.0*sigma
#     print(np.sum(out))
#     if np.sum(out) == np.sum(outliers):
#         break
#     outliers += out

# print(outliers)
sigma = np.std(lc)
outliers = np.abs(lc) > 0.8*sigma
plt.plot(eb.time[outliers], lc[outliers], "x", c="red")

In [ ]:
outliers = tess_cpm.utils.get_outliers(lc, 1)
plt.plot(eb.time, lc)
plt.plot(eb.time[outliers], lc[outliers], "x", c="red")

In [ ]:
eb.holdout_fit_predict(k=10, mask=~outliers);

In [ ]:
eb.plot_pix_by_pix("cpm_subtracted_flux")

In [ ]:
a = eb.get_aperture_lc("rescaled_cpm_subtracted_flux")
plt.plot(eb.time, a)

In [ ]:
l = lk.TessLightCurve(eb.time, a)
l.plot()
pg = l.to_periodogram(oversample_factor=1000)
fig, axs = plt.subplots(2, 1, figsize=(15, 8))
pg.plot(ax=axs[0], c='k')
pg.plot(ax=axs[1], c='k', view='period')
fig.suptitle("Periodogram", fontsize=20, y=0.95)
period = pg.period_at_max_power
print(f"Max Power Period: {period}")
l.fold(period.value*2).scatter()
plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20);

In [ ]:
lc = lk.LightCurve(time=eb.time, flux=a)
pg = lc.to_periodogram(oversample_factor=1000)
fig, axs = plt.subplots(2, 1, figsize=(15, 8))
pg.plot(ax=axs[0], c='k')
pg.plot(ax=axs[1], c='k', view='period')
fig.suptitle("Periodogram", fontsize=20, y=0.95)
period = pg.period_at_max_power
print(f"Max Power Period: {period}")
lc.fold(period.value*2).scatter()
plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20);

In [ ]:
cpm_periodogram(fits_file)

In [ ]:
cpm = tess_cpm.CPM(fits_file, remove_bad=True)

In [ ]:
cpm.set_poly_model(1, 4, 2)
cpm.set_target(50, 50)
# cpm.set_target(16, 29)
# cpm.set_target(34, 71)  # Bright region bottom-right
# cpm.set_target(36, 11)
cpm.set_exclusion(10)
cpm.set_predictor_pixels(256, method='cosine_similarity')

In [ ]:
plt.figure(figsize=(12, 8))
n = 10
# locs, mask = c.get_contributing_pixels(n)
# print(c.rescaled_predictor_pixels_fluxes.shape)
# print(locs[2])
# plt.plot(c.time, c.rescaled_im_fluxes[:, locs[0][0], locs[0][1]], '-')
# plt.plot(c.time, c.rescaled_predictor_pixels_fluxes[:, 4])
for i in range(n):
    plt.figure(figsize=(12, 8))
    t = cpm.rescaled_target_fluxes
    p = cpm.rescaled_predictor_pixels_fluxes[:, i]
    plt.plot(cpm.time, t, '.')
    plt.plot(cpm.time, p, '-', alpha=0.4)
    plt.title(np.dot(p, t)/(np.linalg.norm(p)*np.linalg.norm(t)), fontsize=25)
    plt.show();

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
cpm.entire_image(0.5, rescale=True, polynomials=True)
diff = cpm.im_diff
upscaled_diff = cpm.pixel_medians*cpm.im_diff

fig, axes = plt.subplots(1, 3, figsize=(18, 18))

ims = []
for i in range(0, diff.shape[0], 10):
    im1 = axes[0].imshow(cpm.im_fluxes[i], origin="lower", animated=True,
                        vmin=np.percentile(cpm.im_fluxes[0], 10), vmax=np.percentile(cpm.im_fluxes[0], 90))
    im2 = axes[1].imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    im3 = axes[2].imshow(upscaled_diff[i], origin="lower", animated=True,
                   vmin=np.percentile(upscaled_diff, 1), vmax=np.percentile(upscaled_diff, 99));
    ims.append([im1, im2, im3]);
fig.colorbar(im1, ax=axes[0], fraction=0.046)
fig.colorbar(im2, ax=axes[1], fraction=0.046)
fig.colorbar(im3, ax=axes[2], fraction=0.046)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())


In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
cpm = tess_cpm.CPM(fits_file, remove_bad=True)
cpm.set_poly_model(1, 4, 2)
cpm.set_target(50, 50)
# cpm.set_target(16, 29)
# cpm.set_target(34, 71)  # Bright region bottom-right
# cpm.set_target(36, 11)
cpm.set_exclusion(10)
cpm.set_predictor_pixels(256, method='cosine_similarity')
cpm.xval(0.5)

In [ ]:
# fits_file = "tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"